In [1]:
import pandas as pd
import numpy as np

## Features engineering de conversion

### Carga de datos

In [2]:
data = pd.read_pickle("../data/device_data.pkl")

---

### Cargo features generados hasta el momento


In [3]:
# Trimestre 2 - 3 - 4 para los que tengo datos anteriores
CANT_TRIMESTRES = 4

In [4]:
TRIMESTRES = {}
trim_no_encontrados =[]

In [5]:
def generar_dataframes_ventana(numero_ventana,dataframe):
    
    if numero_ventana == 4 :   #Para la ventana 4 se necesitan los 11 mil usuarios existentes
        df = pd.read_csv("../data/users.csv", index_col = 0)
    
    else:    
        condicion = dataframe["trimestre"] == numero_ventana
        df = dataframe[condicion]["USER_ID"].drop_duplicates().to_frame().set_index("USER_ID").copy()
    
    return df.sort_index()
    

In [6]:
print("Cargando features")
for ventana_nro in range(1,CANT_TRIMESTRES) : #Trimestres del 1 al 3
    try:
        TRIMESTRES[ventana_nro] = pd.read_pickle("../data/features/ventana_{}_devicedata.pkl".format(ventana_nro))
        
    except FileNotFoundError:
        print("No se encontraron los features, se generarán los dataframe requeridos")
        trim_no_encontrados.append(ventana_nro)
    continue
    
for ventana_nro in trim_no_encontrados:    
    TRIMESTRES[ventana_nro] = generar_dataframes_ventana(ventana_nro,data)

Cargando features
No se encontraron los features, se generarán los dataframe requeridos
No se encontraron los features, se generarán los dataframe requeridos
No se encontraron los features, se generarán los dataframe requeridos


### Set para predecir

In [7]:
try:
    print("Cargando features para predecir")
    TRIMESTRES[4] = pd.read_pickle("../data/features/predecir_devicedata.pkl")
except FileNotFoundError:
    print("No se encontro el feature deseado, se generara el dataframe requerido")
    TRIMESTRES[4] = generar_dataframes_ventana(4,data)

Cargando features para predecir
No se encontro el feature deseado, se generara el dataframe requerido


---

### Diccionario de features

In [8]:
#Posicion dentro del diccionario de features que ocupa la lista de columnas generadas por el mismo
POS_COLUMNAS_DEL_FEATURE = 0
POS_TIPO_DATO = 1
POS_DATO_NAN = 2

In [9]:
# Clave = nombre_feature : Valor = funcion
FUNCIONES_FEATURES = {}
# Clave = nombre_feature : Valor = (columnas,tipo_dato,valor_nan)
DATOS_FEATURES = {}

In [10]:
FEATURES_YA_GENERADOS = {}

for ventana_nro in range (1, CANT_TRIMESTRES + 1):
    
    FEATURES_YA_GENERADOS[ventana_nro] = list(TRIMESTRES[ventana_nro].columns.values)

---


---

## Creación de features

#### Vistazo del dataset: __Conversiones__

In [11]:
data.head()

,FEC_EVENT,CONNECTION_SPEED,IS_MOBILE_DEVICE,USER_ID,mes,trimestre
0,2018-01-12 00:00:06,1,False,6574,1,1
1,2018-01-12 00:00:06,3,True,2639,1,1
2,2018-01-12 00:00:08,0,False,3898,1,1
3,2018-01-12 00:00:09,1,False,7996,1,1
4,2018-01-12 00:00:14,1,False,6111,1,1


In [12]:
data.shape

(2871406, 6)

---

#### ¿Cuantas veces entró desde dispositivo movil y cuantas no?

In [13]:
nombre = "vistas trimestre por dispositivo"

In [14]:
def cantidad_de_vistas_en_trimestre_por_dispositivos(dataframe):
    """ Devuelve 2 features 'Vistas trim no MD' y 'Vistas trim MD'
        MD significa Mobile Device
    """
    
    feature = dataframe.groupby(["USER_ID","IS_MOBILE_DEVICE"]).agg({"trimestre":"count"})
    feature.reset_index(inplace = True)
    feature=feature.pivot(index='USER_ID',columns='IS_MOBILE_DEVICE',values='trimestre')

    feature=feature.reindex(columns=[False,True]) #Acomodo las columnas por las dudas que queden en distinto orden
    feature.columns = ['Vistas trim No MD','Vistas trim MD'] #MD = Mobile device
    feature.head()

    return feature

In [15]:
FUNCIONES_FEATURES[nombre] = cantidad_de_vistas_en_trimestre_por_dispositivos

In [16]:
columnas_generadas = ['Vistas trim No MD','Vistas trim MD']
tipo_dato = "int64"
dato_nan = 0

DATOS_FEATURES[nombre] = (columnas_generadas,tipo_dato,dato_nan) 

---

---

## Generacion de features

In [17]:
def feature_ya_generado(nombre_feature, trimestre):
    """Corrobora si un feature ya fue generado """
    
    #Obtengo las columnas que genera ese feature
    columnas = DATOS_FEATURES[nombre_feature][POS_COLUMNAS_DEL_FEATURE]
    for columna in columnas:
        if columna in FEATURES_YA_GENERADOS[trimestre]:
            return True
    return False

In [18]:
def generar_feature_en_ventana(dataframe,trimestre, nombre_feature, destino):
    """Genera un feature en la ventana correspondiente, con los tipos de datos correspondientes"""
    
    #Calculo el feature
    funcion_feature = FUNCIONES_FEATURES[nombre_feature]
    dataframe = funcion_feature( dataframe[dataframe["trimestre"] == trimestre] )
    
    #Uno los features nuevos con los viejos
    dataframe = destino[trimestre].merge(dataframe, left_index = True, right_index= True, how = 'left')
    
    #Rello los nans, si es que los hubiera
    dataframe = dar_formato_al_feature_nuevo(dataframe,nombre_feature)
    
    destino[trimestre] = dataframe

In [19]:
def dar_formato_al_feature_nuevo(dataframe, nombre_feature):
    """Rellena los Nans de un dataframe con features nuevos"""
    
    
    #Datos para rellenar los Nan
    columnas = DATOS_FEATURES[nombre_feature][POS_COLUMNAS_DEL_FEATURE]
    tipo_dato= DATOS_FEATURES[nombre_feature][POS_TIPO_DATO]
    valor_nan = DATOS_FEATURES[nombre_feature][POS_DATO_NAN]
    
    dataframe.fillna(valor_nan, inplace = True)
    dataframe[columnas] = dataframe[columnas].astype(tipo_dato)
    
    return dataframe
    

In [20]:
def generar_features_faltantes(dataframe,destino,trimestre):
    """
    Funcion para generar los features faltantes para un trimestre, 
    evita la repeticion de calculo de features si estos ya se encuentran generados.
    
    dataframe: Dataframe a partir de cual se calcularan los features.
    destino: Diccionario de ventanas para los que calcular los features.
    trimestre: Numero de ventana a partir de la cual se calculan los features.
    """
    
    for nombre_feature in FUNCIONES_FEATURES:    
        if feature_ya_generado(nombre_feature, trimestre):
            continue

        #Si no fueron generados, los genero y actualizo el listado:   
        generar_feature_en_ventana(dataframe,trimestre, nombre_feature, destino)   
        
        #Agrego las columnas nuevas al diccionario
        columnas_generadas = DATOS_FEATURES[nombre_feature][POS_COLUMNAS_DEL_FEATURE]
        FEATURES_YA_GENERADOS[trimestre].extend(columnas_generadas)

In [21]:
for ventana_nro in range(1, CANT_TRIMESTRES + 1):
    generar_features_faltantes(data, TRIMESTRES, ventana_nro)

In [22]:
TRIMESTRES[1].head()

,Vistas trim No MD,Vistas trim MD
USER_ID,,
0,0,190
1,2,51
2,21,26
3,0,80
4,155,3


In [23]:
TRIMESTRES[4].head()

,Vistas trim No MD,Vistas trim MD
USER_ID,,
0,0,112
1,9,50
2,19,55
3,5,52
4,46,2


### Guardado

In [24]:
for ventana_nro in range(1, CANT_TRIMESTRES ):
    TRIMESTRES[ventana_nro].to_pickle("../data/features/ventana_{}_devicedata.pkl".format(ventana_nro))

TRIMESTRES[4].to_pickle("../data/features/predecir_devicedata.pkl")